In [41]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import scipy
import glob
%matplotlib inline

In [16]:
# Create and test the video sequence for the Car4 dataset

vid_seq = []

# Iterate through all the images with .jpg extension
for file in glob.glob('C:\\Users\\shant\\lucas_kanade_tracker\\data\\Car4\\img\\*.jpg'):
    
    #Read the images sequentially and append them in the vid_seq list
    img = cv2.imread(file)
    height, width, layers = img.shape
    size = (width,height)
    vid_seq.append(img)

fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('Car.avi',fourcc,5.0,size)

for i in range(len(vid_seq)):
    out.write(vid_seq[i])
out.release()

In [17]:
# Create and test the video sequence for the Bolt dataset

vid_seq = []

# Iterate through all the images with .jpg extension
for file in glob.glob('C:\\Users\\shant\\lucas_kanade_tracker\\data\\Bolt2\\img\\*.jpg'):
    
    # Read the images sequentially and append them in the vid_seq list
    img = cv2.imread(file)
    height, width, layers = img.shape
    size = (width,height)
    vid_seq.append(img)

fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('Bolt.avi',fourcc,5.0,size)

for i in range(len(vid_seq)):
    out.write(vid_seq[i])
out.release()

In [18]:
# Create and test the video sequence for the BabyDragon dataset

vid_seq = []

# Iterate through all the images with .jpg extension
for file in glob.glob('C:\\Users\\shant\\lucas_kanade_tracker\\data\\DragonBaby\\img\\*.jpg'):
    
    # Read the images sequentially and append them in the vid_seq list
    img = cv2.imread(file)
    height, width, layers = img.shape
    size = (width,height)
    vid_seq.append(img)

fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('DragonBaby.avi',fourcc,5.0,size)

for i in range(len(vid_seq)):
    out.write(vid_seq[i])
out.release()

## Implementation of the Lucas Kanade metod for Affine Warp

Warp the image $I$ to obtain $I(W[x,y];P)$

Compute the error image $T(x)-I(W[x,y];P)$

Warp the Gradient ${\nabla{I}}$ with $(W[x,y];P)$

Evaluate $\frac{\partial W}{\partial P}$ at $([x,y];P)$

Compute Steepest descent images ${\nabla{I}}\frac{\partial W}{\partial P}$

Compute the Hessian Matrix $\sum{({\nabla{I}}\frac{\partial W}{\partial P})}^{T}{{\nabla{I}}\frac{\partial W}{\partial P}}$

Compute $\sum{({\nabla{I}}\frac{\partial W}{\partial P})}{(T(x)-I(W[x,y];P))}$

Compute ${\Delta P}$

$P{\leftarrow}P+{\Delta P}$

Keep Iterating till the magnitude of ${\Delta P}$ is negligible

And also ${\Delta P} = {({A}^{T}{A}})^{-1}{A^{T}}{b}$

Here ${{A}^{T}{A}}$ is the Hessian Matrix that has been computed above and $A$ is the steepest descent image and $b$ is basicallty the error image that has been computed in step 2

Affine Transform is given as ${\begin{bmatrix} 1+{p_1} & {p_3} & {p_5}\\ {p_2} & {1+{p_4}} & {p_{6}}\\0&0&1\\\end{bmatrix}}$

In [11]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from scipy.ndimage import affine_transform
import glob
%matplotlib inline

# Get a grayscale image to input in the Lucas Kanade 
def get_grayscale_image(image):
    
    """
    Inputs:
    path_to_frame: The path to the frame that needs to be converted to a grayscale image: dtype=string
    
    """
    grayscale = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    return grayscale

# Reference: CMU Graduate computer Vision Course
def crop_warped(image,rect):
    
    """
    Inputs:
    image: The warped image after performing an affine transformation
    rect: The upleft and the truth bounding box
    
    """
    #warped = image[rect[1]:rect[1]+rect[3],rect[0]:rect[0]+rect[2]]
    warped = image[rect[1]:rect[3], rect[0]:rect[2]]
    return warped
    
def LucasKanadeAffine(image,template,rect,parameters,threshold, iterations):
    
    """
    Inputs:
    
    image: The input gray scale image
    template: The template image
    rect: The top left coordinates and the bottom right coordinates of the rectangle format--> [x1,y1,x2,y2]
    threshold: The threshold that has been set to terminate the iterations as soon as delta p value approaches the threshold value
    parameters: The initial parameters of the affine warp
    iterations: Number of times the algorithm needs to run
    
    Returns:
    p: Paramters of the affine warp matrix
    Upper left bounding bax coordinates
    Downright bounding box coordinates
    """
    # Crop the region of interest from the template frame
    template = crop_warped(template,rect)
    
    # Parameters of the affine matrix
    p = np.array([parameters]).reshape(-1,1)
    
    # Threshold for convergence
    thresh = threshold
    
    # Initial Affine Matrix
    affine_matrix = np.array([[1.0,0.0,0.0],[0.0,1.0,0.0]])
    
    I = affine_matrix
    
    for iterations in range(iterations):
        
        # Warp the input image
        warped_image = cv2.warpAffine(image,affine_matrix,image.shape)
        
        # Compute the error image
        error_image = template - crop_warped(warped_image,rect)
        
        # Compute the image gradient in the x direction
        sobelx = cv2.Sobel(image,cv2.CV_64F, dx = 1, dy = 0, ksize = 3)
        
        # Crop the Region of interest in our image gradient
        sobel_x_cropped = crop_warped(sobelx,rect)
        
        # Compute the warped gradient in the x direction
        sobel_x_warped = cv2.warpAffine(sobel_x_cropped, affine_matrix, sobel_x_cropped.shape)
        
        # Compute the image gradient in the y direction
        sobely = cv2.Sobel(image,cv2.CV_64F, dx = 0, dy = 1, ksize = 3)
        
        # Crop the Region of interest in our image gradient
        sobel_y_cropped = crop_warped(sobely,rect)
        
        # Compute the warped gradient in the y direction
        sobel_y_warped = cv2.warpAffine(sobel_y_cropped, affine_matrix, sobel_y_cropped.shape)
        
        # Flatten out the warped gradients
        sobel_x_warped = sobel_x_warped.reshape(-1,1)
        sobel_y_warped = sobel_y_warped.reshape(-1,1)
        
        # Warp the image gradient with the warping function to produce an mx2 matrix
        # Horizontally stack the derivatives
        gradient_image_warped = np.hstack((sobel_x_warped,sobel_y_warped))
        
        # Evaluate the Jacobian and the steepest descent 
        count = 0
        steepest_descent = []
        for y in range(rect[0],rect[2]):
            for x in range(rect[1],rect[3]):
                
                Jacobian = [x*sobel_x_warped[count][0], x*sobel_y_warped[count][0], y*sobel_x_warped[count][0], y*sobel_y_warped[count][0], sobel_x_warped[count][0], sobel_y_warped[count][0]]
                steepest_descent.append(Jacobian)
                count = count + 1
        steepest_descent = np.array(steepest_descent)
        
        # Calculate the Hessian and inverse hessian from the steepest descent
        sd_params_update = np.dot(steepest_descent.T,error_image.reshape(-1,1))
        hessian = np.dot(steepest_descent.T,steepest_descent)
        hessian_inverse = np.linalg.inv(hessian)
        
        # Update delta p using solution for least squares
        delta_p = np.dot(hessian_inverse, sd_params_update)
        
        # Update the parameters p
        p = p + delta_p
        print(f'After Iteration: {iterations} Value of p: {p}')
        
        # Update the affine matrix with the newly generated parameters
        affine_matrix = p.reshape(2,3) + I
        
        # Convergence test
        if np.linalg.norm(delta_p) <= thresh:
            break
    
    top_left_coordinates = np.array([[rect[1]],[rect[0]],[1]])
    bottom_right_coordinates = np.array([[rect[1] + rect[3]], [rect[0]+rect[2]], [1]])
    updated_top_left_coordinates = np.dot(affine_matrix, top_left_coordinates)
    updated_bottom_right_coordinates = np.dot(affine_matrix, bottom_right_coordinates)
    
    return p, updated_top_left_coordinates, updated_bottom_right_coordinates

In [12]:
# Reading in the frames and performing the tracking
# Take an initial bounding box for the car, dragon baby and bolt4

rect_car = np.array([70,51,107+70,87+51])
rect_bolt = np.array([269,75,34+269,64+75])
rect_baby = np.array([160,83,56+160,65+83])

# Create and test the video sequence for the Bolt dataset

files  = glob.glob('C:\\Users\\shant\\lucas_kanade_tracker\\data\\Bolt2\\img\\*.jpg')
template = cv2.imread(files[0])
grayscale_template = get_grayscale_image(template)

# grayscale template image

# The fourcc code
fourcc = cv2.VideoWriter_fourcc(*'XVID')

# Video writer object
out = cv2.VideoWriter('Bolt.avi',fourcc,20.0, (480, 270))

# List to append the frame sequences
vid_seq = []

# Iterate through all the images with .jpg extension
for file in files:
    
    image = cv2.imread(file)
    image_copy = image.copy()
    
    # Input the gray scale image
    grayscale_image = get_grayscale_image(image_copy)
    
    # Run the Lucas Kanade Algorithm
    paramters, top_left_coordinates, bottom_right_coordinates = LucasKanadeAffine(grayscale_image,grayscale_template,rect_bolt,parameters=[0.0,0.0,0.0,0.0,0.0,0.0],threshold = 0.001, iterations = 20)
    
    # Create a bounding box
    image = cv2.rectangle(image,(int(top_left_coordinates[0][0]),int(top_left_coordinates[1][0])), (int(bottom_right_coordinates[0][0]), int(bottom_right_coordinates[1][0])), (0, 0, 255), 2)
    
    vid_seq.append(image)
    
for i in range(len(vid_seq)):
    out.write(vid_seq[i])
out.release()

After Iteration: 0 Value of p: [[ 4.89727119e-03]
 [ 3.85874728e-03]
 [-7.37205627e-04]
 [ 4.12810286e-03]
 [-2.95140180e-01]
 [-1.62285309e+00]]
After Iteration: 1 Value of p: [[ 2.45368498e-02]
 [ 9.38192965e-03]
 [ 2.55157555e-03]
 [-1.45091837e-03]
 [-3.00411055e+00]
 [-5.96729701e-01]]
After Iteration: 2 Value of p: [[-2.90311218e-02]
 [ 2.97400391e-02]
 [-2.20992115e+00]
 [-5.99074114e-01]
 [ 5.96030899e+02]
 [ 1.57999452e+02]]
After Iteration: 3 Value of p: [[-1.24762486e-01]
 [ 3.17400747e-02]
 [-2.19758506e+00]
 [-6.02685198e-01]
 [ 6.00052715e+02]
 [ 1.58617791e+02]]
After Iteration: 4 Value of p: [[-2.52526552e-01]
 [ 2.68688009e-02]
 [-2.20189259e+00]
 [-6.04631693e-01]
 [ 6.11437969e+02]
 [ 1.59335167e+02]]
After Iteration: 5 Value of p: [[-3.96078660e-01]
 [ 1.81814481e-02]
 [-2.19948384e+00]
 [-6.05163741e-01]
 [ 6.22100713e+02]
 [ 1.59929895e+02]]
After Iteration: 6 Value of p: [[-5.94232218e-01]
 [-9.03183174e-03]
 [-2.20188014e+00]
 [-6.07471796e-01]
 [ 6.38133011e+02

LinAlgError: Singular matrix

In [143]:
frame1 = cv2.imread('C:\\Users\\shant\\lucas_kanade_tracker\\data\\Bolt2\\img\\0001.jpg',0)
frame2 = cv2.imread('C:\\Users\\shant\\lucas_kanade_tracker\\data\\Car4\\img\\0001.jpg',0)
frame3 = cv2.imread('C:\\Users\\shant\\lucas_kanade_tracker\\data\\DragonBaby\\img\\0001.jpg',0)
H, W = template.shape
Jx = np.tile(np.linspace(0, W-1, W), (H, 1)).flatten().T
Jy = np.tile(np.linspace(0, H-1, H), (W, 1)).T.flatten()
x = 70
y = 51
length = 87
width = 107
rect = [70,51,107,87]
frame2_copy = frame2.copy()
frame2_copy = cv2.rectangle(frame2_copy,(x,y),(x+width,y+length),color = (255,0,0),thickness = 2)
template = frame2_copy[rect[1]:rect[1]+rect[3],rect[0]:rect[0]+rect[2]]
cv2.imshow('frame 2', frame2_copy)
plt.imshow(template)

cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)
template.shape[0]

87

In [206]:
files  = glob.glob('C:\\Users\\shant\\lucas_kanade_tracker\\data\\Bolt2\\img\\*.jpg')
grayscale_template = get_grayscale_image(files[0])

cv2.imshow('template',grayscale_template)

cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)

-1

In [222]:
grayscale_template.shape[2]

IndexError: tuple index out of range

In [9]:
x = np.array([[1,2,3],[4,5,6],[7,8,9]]).reshape(-1,1)
x

array([[1],
       [2],
       [3],
       [4],
       [5],
       [6],
       [7],
       [8],
       [9]])